In [1]:
import lzma
import json
import pandas as pd
import numpy as np
import re
import glob
import os
import sys
import multiprocessing as mp
import pickle

In [3]:
os.chdir(r"C:\Users\erica\OneDrive - University of Southern California\Projects_at_USC\CKIDS_current\data\loc_data_3")

In [4]:
os.getcwd()

'C:\\Users\\erica\\OneDrive - University of Southern California\\Projects_at_USC\\CKIDS_current\\data\\loc_data_3'

In [5]:
# lists of file names
# json xz
json_files = glob.glob("*_UTC.json")
# descrip txts
txt_files = glob.glob("*_UTC.txt")
# location txts
loc_files = glob.glob("*_location.txt")
# comments jsons
cmt_files = glob.glob("*_comments.json")
# num of jpgs
jpg_files = glob.glob("*.jpg")



print(len(json_files))
print(len(txt_files))
print(len(loc_files))
print(len(cmt_files))
print(len(jpg_files))

250
217
251
106
424


In [7]:
# Add all json files to a dataframe
temp = pd.DataFrame()

path_to_json = r'C:\Users\erica\OneDrive - University of Southern California\Projects_at_USC\CKIDS_current\data\loc_data_3' 

json_pattern = os.path.join(path_to_json,'*_UTC.json')
file_list = glob.glob(json_pattern)

for file in json_files:
    # print(file)
    data = pd.read_json(file, orient=str)
    postnode = data['node']
    temp = temp.append(postnode, ignore_index = True)

In [8]:
temp.shape

(250, 50)

In [9]:
temp.head(3)

,__typename,accessibility_caption,caption_is_edited,commenting_disabled_for_viewer,comments_disabled,dimensions,display_resources,display_url,edge_liked_by,edge_media_preview_comment,...,viewer_in_photo_of_you,dash_info,encoding_status,is_published,product_type,title,video_duration,video_url,video_view_count,edge_sidecar_to_children
0,GraphImage,Photo by Francisco Bugarin in Umami Burger. Im...,0.0,0.0,0.0,"{'height': 810, 'width': 1080}","[{'config_height': 480, 'config_width': 640, '...",https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 14},"{'count': 0, 'edges': []}",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GraphImage,Photo by Panterra Caraway in Umami Burger.,0.0,0.0,0.0,"{'height': 1080, 'width': 1080}","[{'config_height': 640, 'config_width': 640, '...",https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 10},"{'count': 0, 'edges': []}",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GraphImage,Photo by Anaheim Packing District in Umami Bur...,1.0,0.0,0.0,"{'height': 1080, 'width': 1080}","[{'config_height': 640, 'config_width': 640, '...",https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 332},"{'count': 0, 'edges': []}",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
json_fnames = [f.split('.json')[0] for f in json_files]
json_fnamesS = pd.Series(json_fnames)
temp['file_name'] = json_fnamesS

In [11]:
temp2 = temp[temp['__typename'] == "GraphImage"]
print(temp2.shape)

(167, 51)


In [12]:
# temp2.columns

In [13]:
# temp2.iloc[0]

In [14]:
df = temp2[['file_name', 'accessibility_caption', 'display_url', 'edge_liked_by', 'id', 'is_ad', 'location', 'owner', 'taken_at_timestamp']]

In [15]:
df.head()

,file_name,accessibility_caption,display_url,edge_liked_by,id,is_ad,location,owner,taken_at_timestamp
0,2020-02-01_20-20-34_UTC,Photo by Francisco Bugarin in Umami Burger. Im...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 14},2234460196022214179,0.0,"{'has_public_page': True, 'id': 142724889, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580588e+09
1,2020-02-02_01-20-04_UTC,Photo by Panterra Caraway in Umami Burger.,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 10},2234610938626187438,0.0,"{'has_public_page': True, 'id': 214098779, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580606e+09
2,2020-02-03_01-00-35_UTC,Photo by Anaheim Packing District in Umami Bur...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 332},2235325885460691420,0.0,"{'has_public_page': True, 'id': 19002234, 'lat...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580692e+09
3,2020-02-03_02-03-10_UTC,Photo by 100% VEGAN FAMILY in Umami Burger wit...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 46},2235357414060560846,0.0,"{'has_public_page': True, 'id': 19002234, 'lat...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580695e+09
4,2020-02-03_23-41-46_UTC,Photo by Rosendo Herrera in Umami Burger. Imag...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 11},2236011019983973241,0.0,"{'has_public_page': True, 'id': 321424183, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580773e+09


In [16]:
df.to_csv("post_jsons_df_3.csv")

In [40]:
# Read in text descrip files (hashtags)

In [17]:
def read_txt(file):
    text = open(file, "r", encoding="utf8").read()
    return text   # return as string


def get_ht(text):
    ht = re.findall(r"#(\w+)", text)
    return ht

In [18]:
texts = []

for f in txt_files:
    text = open(f, 'r', encoding="utf8").read()
    texts.append(text) 

In [19]:
texts = list(map(read_txt, txt_files))
hashtags = list(map(get_ht, texts))

In [20]:
# get file names
txt_fnames = [f.split('.txt')[0] for f in txt_files]

# convert all to series, and concat them to a df
textsS = pd.Series(texts)
hashtagsS = pd.Series(hashtags)
txt_fnamesS = pd.Series(txt_fnames)

df_txt = pd.DataFrame(pd.concat([txt_fnamesS, textsS, hashtagsS], axis=1))
df_txt.columns = ['file_name', 'descrip', 'hashtags']

In [21]:
df_txt.to_csv("df_txt.csv")

In [ ]:
# Join the json df and txt df by their common filename

In [22]:
print(df_txt.columns)
print(df.columns)

Index(['file_name', 'descrip', 'hashtags'], dtype='object')
Index(['file_name', 'accessibility_caption', 'display_url', 'edge_liked_by',
       'id', 'is_ad', 'location', 'owner', 'taken_at_timestamp'],
      dtype='object')


In [23]:
df_txt['file_name'].head()

0    2020-02-02_01-20-04_UTC
1    2020-02-03_01-00-35_UTC
2    2020-02-03_02-03-10_UTC
3    2020-02-03_23-41-46_UTC
4    2020-02-06_02-51-43_UTC
Name: file_name, dtype: object

In [25]:
df['file_name'].head()

0    2020-02-01_20-20-34_UTC
1    2020-02-02_01-20-04_UTC
2    2020-02-03_01-00-35_UTC
3    2020-02-03_02-03-10_UTC
4    2020-02-03_23-41-46_UTC
Name: file_name, dtype: object

In [26]:
# merge the post metadata (jsons) df and text descrip df (txt)
merged = pd.merge(df, df_txt, on='file_name')
print(merged.shape)

(148, 11)


In [27]:
merged.head()

,file_name,accessibility_caption,display_url,edge_liked_by,id,is_ad,location,owner,taken_at_timestamp,descrip,hashtags
0,2020-02-02_01-20-04_UTC,Photo by Panterra Caraway in Umami Burger.,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 10},2234610938626187438,0.0,"{'has_public_page': True, 'id': 214098779, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580606e+09,Yummy dinner😋🍔🍟\n#impossibleburger\n,[impossibleburger]
1,2020-02-03_01-00-35_UTC,Photo by Anaheim Packing District in Umami Bur...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 332},2235325885460691420,0.0,"{'has_public_page': True, 'id': 19002234, 'lat...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580692e+09,We've got stars in our eyes for The UFO desser...,"[letsdodessert, packingdistrict, anaheim, ocfo..."
2,2020-02-03_02-03-10_UTC,Photo by 100% VEGAN FAMILY in Umami Burger wit...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 46},2235357414060560846,0.0,"{'has_public_page': True, 'id': 19002234, 'lat...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580695e+09,Who else loves the impossible patty?!?😍 . -\n-...,"[vegan, veganfood, veganismo, veganfamily, veg..."
3,2020-02-03_23-41-46_UTC,Photo by Rosendo Herrera in Umami Burger. Imag...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 11},2236011019983973241,0.0,"{'has_public_page': True, 'id': 321424183, 'la...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580773e+09,#umamiburger #impossibleburger\n,"[umamiburger, impossibleburger]"
4,2020-02-06_02-51-43_UTC,Photo by Willerm De Passé Frekan in Umami Burg...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,{'count': 35},2237556175943954701,0.0,"{'has_public_page': True, 'id': 19002234, 'lat...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580958e+09,Back at it again ! My favorite burger joint!! ...,[]


In [28]:
merged.to_csv("posts_merged_df_3.csv")